# Eleringi API andmete laadimine

See notebook laeb tunnipõhised tarbimisandmed Eleringi API-st alates 2025-10-17 kuni tänaseni ja salvestab need CSV-faili.

In [2]:
# 1. Import Required Libraries
import requests
import pandas as pd
from datetime import datetime

## 2. Määra Eleringi API lõpp-punkt ja parameetrid
Määrame API URL-i ja päringu parameetrid (algus- ja lõppkuupäev, intervall).

In [3]:
# 2. Set Elering API Endpoint and Parameters
url = "https://dashboard.elering.ee/api/nps/production-consumption"
params = {
    "start": "2025-10-17T00:00:00.000Z",
    "end": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.000Z"),
    "interval": "hour"
}

C:\Users\tarmo\AppData\Local\Temp\ipykernel_31024\2055854956.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "end": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.000Z"),


## 3. Laadi andmed Eleringi API-st
Teeme GET-päringu ja kontrollime vastust.

In [4]:
# 3. Fetch Data from Elering API
response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    print(f"Andmed edukalt laetud, kirjeid: {len(data['data'])}")
else:
    print(f"Viga API päringus: {response.status_code}")

Viga API päringus: 500


## 4. Andmete parsimine ja salvestamine
Laeme saadud andmed pandas DataFrame'i, kuvame esimesed read ja salvestame CSV-faili.

In [8]:
# 4.1 Lae forecasti fail (poolitaja ;, koma kümnendik)
forecast_path = "scripts/output/forecast_consumption_hourly_next7_tallinn_20250918_20250924corr.csv"
df_forecast = pd.read_csv(
    forecast_path,
    sep=';',
    decimal=',',
    skipinitialspace=True,
    encoding='cp1257'  # Või 'latin1' kui cp1257 ei tööta
)

# 4.2 Forecasti ajatempeli UTC-ks ja tunnipõhiseks
import pytz
dt_forecast = pd.to_datetime(df_forecast['datetime_local'], utc=False)
if dt_forecast.dt.tz is None:
    dt_forecast = dt_forecast.dt.tz_localize('Europe/Tallinn')
dt_forecast_utc = dt_forecast.dt.tz_convert('UTC')
df_forecast['datetime_hour'] = dt_forecast_utc.dt.floor('h')

# 4.3 Lae tegeliku tarbimise fail (poolitaja ,)
actuals_path = "scripts/elering_actual_from_20250917.csv"
df_actual = pd.read_csv(actuals_path, sep=',', decimal=',', skipinitialspace=True, encoding='utf-8')
df_actual['datetime_hour'] = pd.to_datetime(df_actual['Ajatempel (UTC)'].astype(float), unit='s', utc=True).dt.floor('h')

# 4.4 Puhasta ja teisenda tarbimise veerud numbriks, tee mõlemad datetime_hour veerud timezone-naive'iks, ühenda ja arvuta viga
df_actual['Tarbimine'] = (
    df_actual['Tarbimine']
    .astype(str)
    .str.replace('"', '')
    .str.replace(',', '.')
    .str.strip()
)
df_actual['Tarbimine'] = pd.to_numeric(df_actual['Tarbimine'], errors='coerce')
df_forecast['consumption_hourly'] = (
    df_forecast['consumption_hourly']
    .astype(str)
    .str.replace('"', '')
    .str.replace(',', '.')
    .str.strip()
)
df_forecast['consumption_hourly'] = pd.to_numeric(df_forecast['consumption_hourly'], errors='coerce')
# Tee mõlemad datetime_hour veerud timezone-naive'iks
if hasattr(df_actual['datetime_hour'].dt, 'tz'):
    df_actual['datetime_hour'] = df_actual['datetime_hour'].dt.tz_localize(None)
if hasattr(df_forecast['datetime_hour'].dt, 'tz'):
    df_forecast['datetime_hour'] = df_forecast['datetime_hour'].dt.tz_localize(None)
df_merge = pd.merge(df_actual, df_forecast, on='datetime_hour', suffixes=('_actual', '_forecast'))
df_merge['viga'] = df_merge['Tarbimine'] - df_merge['consumption_hourly']
df_merge['viga_protsent'] = 100 * df_merge['viga'] / df_merge['Tarbimine']
print(df_merge[['datetime_hour', 'Tarbimine', 'consumption_hourly', 'viga', 'viga_protsent']].head())
# Salvesta tulemus CSV-faili, et saaks Excelis uurida
df_merge.to_csv("forecast_vs_actual_comparison_hourly.csv", index=False)
print("Võrdlustabel salvestatud: forecast_vs_actual_comparison_hourly.csv")

        datetime_hour  Tarbimine  consumption_hourly       viga
0 2025-09-17 21:00:00      712.1          710.404740   1.695260
1 2025-09-17 22:00:00      681.7          675.017802   6.682198
2 2025-09-17 23:00:00      668.8          659.912795   8.887205
3 2025-09-18 00:00:00      676.0          658.397113  17.602887
4 2025-09-18 01:00:00      678.7          663.745422  14.954578
Võrdlustabel salvestatud: forecast_vs_actual_comparison_hourly.csv
